In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

from PIL import Image
from tqdm.notebook import tqdm
from datetime import datetime
from datasets import load_dataset
from torch.cuda.amp import GradScaler, autocast

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

dataset = load_dataset("Teklia/IAM-line")["train"]
os.makedirs("temp_images", exist_ok=True)

data, vocab, max_len = [], set(), 0
for idx, example in tqdm(enumerate(dataset), total=len(dataset)):
    image = example["image"]
    label = example["text"]
    image_path = os.path.join("temp_images", f"{idx}.png")
    image.save(image_path)
    vocab.update(label)
    max_len = max(max_len, len(label))
    data.append([image_path, label])


Using device: cuda


  0%|          | 0/6482 [00:00<?, ?it/s]

In [2]:
class ModelConfigs:
    def __init__(self):
        timestamp = datetime.strftime(datetime.now(), "%Y%m%d%H%M")
        self.model_path = os.path.join("Models/CRNN", timestamp)
        os.makedirs(self.model_path, exist_ok=True)
        self.vocab = "".join(sorted(vocab))
        self.height = 64
        self.max_width = 2048
        self.max_text_length = max_len
        self.batch_size = 4
        self.learning_rate = 0.0003
        self.train_epochs = 50
        self.device = device

configs = ModelConfigs()


In [3]:
class ResizeKeepAspect:
    def __init__(self, height, max_width=None):
        self.height = height
        self.max_width = max_width

    def __call__(self, img):
        w, h = img.size
        new_w = min(int(w * (self.height / h)), self.max_width)
        return img.resize((new_w, self.height), Image.BILINEAR)

augmentation = transforms.RandomApply([
    transforms.RandomRotation(5),
    transforms.RandomPerspective(0.2, p=1.0),
    transforms.ColorJitter(0.3, 0.3)
], p=0.5)

transform = transforms.Compose([
    transforms.Grayscale(1),
    augmentation,
    ResizeKeepAspect(configs.height, configs.max_width),
    transforms.ToTensor(),
])

class IAMDataset(Dataset):
    def __init__(self, data, vocab, transform=None):
        self.data = data
        self.vocab = vocab
        self.transform = transform
        self.char_to_idx = {ch: i for i, ch in enumerate(vocab)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, text = self.data[idx]
        image = Image.open(image_path).convert("L")
        if self.transform:
            image = self.transform(image)
        label = torch.tensor([self.char_to_idx[ch] for ch in text], dtype=torch.long)
        return image, label, len(label)

def collate_fn(batch):
    images, labels, lengths = zip(*batch)
    max_w = max(img.shape[2] for img in images)
    images = [F.pad(img, (0, max_w - img.shape[2])) for img in images]
    images = torch.stack(images)
    max_len = max(lengths)
    padded_labels = torch.full((len(labels), max_len), -1, dtype=torch.long)
    for i, label in enumerate(labels):
        padded_labels[i, :len(label)] = label
    return images, padded_labels, torch.tensor(lengths)


In [4]:
split = int(0.9 * len(data))
train_loader = DataLoader(IAMDataset(data[:split], configs.vocab, transform), batch_size=configs.batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(IAMDataset(data[split:], configs.vocab, transform), batch_size=configs.batch_size, shuffle=False, collate_fn=collate_fn)

class CRNN(nn.Module):
    def __init__(self, vocab_size, hidden=512, dropout=0.3):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.Dropout(dropout),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.Dropout(dropout),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.Dropout(dropout),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.Dropout(dropout),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.Dropout(dropout),
            nn.Conv2d(256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.Dropout(dropout),
        )
        self.lstm = nn.LSTM(input_size=256 * (configs.height // 4), hidden_size=hidden,
                            num_layers=2, bidirectional=True, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden * 2, vocab_size + 1)

    def forward(self, x):
        x = self.conv(x)
        b, c, h, w = x.size()
        x = x.permute(0, 3, 1, 2).reshape(b, w, c * h)
        x, _ = self.lstm(x)
        return self.fc(x).permute(1, 0, 2)

model = CRNN(len(configs.vocab)).to(configs.device)
criterion = nn.CTCLoss(blank=len(configs.vocab))
optimizer = optim.Adam(model.parameters(), lr=configs.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)
scaler = GradScaler()


C:\Users\mh738\anaconda3\envs\gpu\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\mh738\AppData\Local\Temp\ipykernel_14152\1423130349.py:33: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [5]:
def train(model, train_loader, val_loader, configs):
    best_val = float("inf")
    for epoch in range(1, configs.train_epochs + 1):
        model.train()
        total_loss = 0
        for images, labels, label_lens in tqdm(train_loader, desc=f"Epoch {epoch}"):
            images, labels = images.to(configs.device), labels.to(configs.device)
            label_lens = label_lens.to(configs.device)
            optimizer.zero_grad(set_to_none=True)
            with autocast():
                output = model(images)
                input_lens = torch.full((output.size(1),), output.size(0), dtype=torch.long).to(configs.device)
                log_probs = F.log_softmax(output, dim=2)
                loss = criterion(log_probs, labels, input_lens, label_lens)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
        print(f"[Epoch {epoch}] Train Loss: {total_loss / len(train_loader):.4f}")

        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = 0
            for images, labels, label_lens in val_loader:
                images, labels = images.to(configs.device), labels.to(configs.device)
                label_lens = label_lens.to(configs.device)
                output = model(images)
                input_lens = torch.full((output.size(1),), output.size(0), dtype=torch.long).to(configs.device)
                log_probs = F.log_softmax(output, dim=2)
                loss = criterion(log_probs, labels, input_lens, label_lens)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f"[Epoch {epoch}] Val Loss: {val_loss:.4f}")
        scheduler.step(val_loss)
        if val_loss < best_val:
            best_val = val_loss
            torch.save(model.state_dict(), os.path.join(configs.model_path, "best_model.pth"))
            print("✅ Best model saved.")

train(model, train_loader, val_loader, configs)


Epoch 1:   0%|          | 0/1459 [00:00<?, ?it/s]

C:\Users\mh738\AppData\Local\Temp\ipykernel_14152\3538403117.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


[Epoch 1] Train Loss: 3.0983
[Epoch 1] Val Loss: 2.3975
✅ Best model saved.


Epoch 2:   0%|          | 0/1459 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 1.6307
[Epoch 2] Val Loss: 1.2358
✅ Best model saved.


...

[Epoch 48] Train Loss: 0.1783
[Epoch 48] Val Loss: 0.4230
✅ Best model saved.


Epoch 49:   0%|          | 0/1459 [00:00<?, ?it/s]

[Epoch 49] Train Loss: 0.1808
[Epoch 49] Val Loss: 0.4573


Epoch 50:   0%|          | 0/1459 [00:00<?, ?it/s]

[Epoch 50] Train Loss: 0.1776
[Epoch 50] Val Loss: 0.4368


In [6]:
def predict_image(image_path, model, configs, transform):
    model.eval()
    idx_to_char = {i: ch for i, ch in enumerate(configs.vocab)}
    blank_idx = len(configs.vocab)

    image = Image.open(image_path).convert("L")
    image = transform(image)
    image = image.unsqueeze(0).to(configs.device)  # (1, 1, H, W)

    with torch.no_grad():
        output = model(image)
        log_probs = F.log_softmax(output, dim=2)
        pred = torch.argmax(log_probs, dim=2)  # (T, 1)
        pred = pred.squeeze(1).tolist()

        decoded = []
        prev_token = None
        for t in pred:
            if t != blank_idx and t != prev_token:
                decoded.append(idx_to_char[t])
            prev_token = t
        return "".join(decoded)


In [12]:
# Rebuild and load model
loaded_model = CRNN(len(configs.vocab)).to(configs.device)
loaded_model.load_state_dict(torch.load(os.path.join(configs.model_path, "best_model.pth"), map_location=configs.device))
print("✅ Model loaded.")

# Predict an image
test_image_path = r"C:\Users\mh738\Downloads\images (2).png"  # Or path to your own image
prediction = predict_image(test_image_path, loaded_model, configs, transform)
print(f"📝 Predicted Text: {prediction}")


✅ Model loaded.
📝 Predicted Text: Family


In [29]:
# TorchScript export
example_input = torch.randn(1, 1, configs.height, 512).to(configs.device)
traced_model = torch.jit.trace(loaded_model, example_input)
torchscript_path = os.path.join(configs.model_path, "model_scripted.pt")
traced_model.save(torchscript_path)
print(f"TorchScript model saved at: {torchscript_path}")


TorchScript model saved at: Models/CRNN\202506090621\model_scripted.pt


In [35]:
# ONNX export
onnx_path = os.path.join(configs.model_path, "model.onnx")
torch.onnx.export(
    loaded_model, 
    example_input, 
    onnx_path,
    input_names=["input"], 
    output_names=["output"], 
    dynamic_axes={"input": {0: "batch_size", 3: "width"}},
    opset_version=11
)
print(f"ONNX model saved at: {onnx_path}")


ONNX model saved at: Models/CRNN\202506090621\model.onnx


In [42]:
import torch
from PIL import Image
import torchvision.transforms as transforms

# Load TorchScript model
model = torch.jit.load(r"Models/CRNN\202506090621\model_scripted.pt")

model.eval()

# Define the transform same as used in training
transform = transforms.Compose([
    transforms.Grayscale(1),
    transforms.Resize((64, 512)),  # Or ResizeKeepAspect if needed
    transforms.ToTensor()
])

def predict_image(image_path):
    image = Image.open(image_path).convert("L")
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(image)
    pred = output.argmax(dim=2).squeeze().tolist()
    return pred  # decode this to characters using your vocab


In [13]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import torch
import torchvision.transforms as transforms

# Load TorchScript model
model_path = r"Models/CRNN/202506090621/model_scripted.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    model = torch.jit.load(model_path, map_location=device)
    model.eval()
    print("✅ Model loaded")
except Exception as e:
    print("❌ Error loading model:", e)
    exit()

# Set vocabulary (same used during training)
vocab = " !\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
idx_to_char = {i: ch for i, ch in enumerate(vocab)}
blank_idx = len(vocab)

# Transform
transform = transforms.Compose([
    transforms.Grayscale(1),
    transforms.Resize((64, 512)),
    transforms.ToTensor()
])

# Predict
def predict_image(path):
    image = Image.open(path).convert("L")
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        log_probs = torch.nn.functional.log_softmax(output, dim=2)
        pred = torch.argmax(log_probs, dim=2).squeeze().tolist()

        decoded = []
        prev = -1
        for i in pred:
            if i != blank_idx and i != prev:
                decoded.append(idx_to_char.get(i, ""))
            prev = i
        return "".join(decoded)

# GUI
class OCRApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Handwriting OCR App")
        self.root.geometry("800x700")
        self.root.configure(bg="white")

        self.image_path = None
        self.predicted_text = ""

        self.title = tk.Label(root, text="🖋️ Handwriting OCR", font=("Arial", 28, "bold"), bg="white")
        self.title.pack(pady=20)

        self.canvas = tk.Label(root, bg="white")
        self.canvas.pack()

        self.upload_btn = tk.Button(root, text="📁 Upload Image", command=self.upload, font=("Arial", 14), width=20)
        self.upload_btn.pack(pady=10)

        self.predict_btn = tk.Button(root, text="🔍 Predict Text", command=self.predict, font=("Arial", 14), bg="#ccffcc", width=20)
        self.predict_btn.pack(pady=5)

        self.result = tk.Label(root, text="", font=("Courier New", 18), wraplength=700, justify="center", bg="white", fg="#222")
        self.result.pack(pady=20)

        self.export_btn = tk.Button(root, text="📤 Export to File", command=self.export_to_file, font=("Arial", 12), bg="#e0e0ff", width=18)
        self.export_btn.pack(pady=5)

        self.copy_btn = tk.Button(root, text="📋 Copy to Clipboard", command=self.copy_to_clipboard, font=("Arial", 12), bg="#ffd9b3", width=18)
        self.copy_btn.pack(pady=5)

    def upload(self):
        path = filedialog.askopenfilename(filetypes=[("Image files", "*.png *.jpg *.jpeg")])
        if path:
            self.image_path = path
            img = Image.open(path)
            img = img.resize((400, 100))
            self.tk_img = ImageTk.PhotoImage(img)
            self.canvas.configure(image=self.tk_img)
            self.result.config(text="")
            self.predicted_text = ""

    def predict(self):
        if not self.image_path:
            self.result.config(text="❌ Please upload an image first.")
            return

        try:
            prediction = predict_image(self.image_path)
            self.predicted_text = prediction
            self.result.config(text=f"📝 Predicted Text:\n\n{prediction}")
        except Exception as e:
            print("❌ Prediction Error:", e)
            self.result.config(text="❌ An error occurred. Check the console.")

    def export_to_file(self):
        if not self.predicted_text:
            messagebox.showinfo("Info", "No text to export.")
            return
        file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text Files", "*.txt")])
        if file_path:
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(self.predicted_text)
            messagebox.showinfo("Success", "Text exported successfully!")

    def copy_to_clipboard(self):
        if not self.predicted_text:
            messagebox.showinfo("Info", "No text to copy.")
            return
        self.root.clipboard_clear()
        self.root.clipboard_append(self.predicted_text)
        self.root.update()
        messagebox.showinfo("Copied", "Text copied to clipboard!")

# Launch
if __name__ == "__main__":
    root = tk.Tk()
    app = OCRApp(root)
    root.mainloop()


✅ Model loaded
